In [2]:
import pandas as pd
import numpy as np

from controllers.token import Token
from controllers.titlequery import TitleQuery

In [3]:
token = Token()

In [6]:
token.get_token()


'g-jSOa2OZX9OgW5eCrtw_ZRGN3e4px0iQoR_x2egPsrNm8NnaY84OJ2id5FE50vCxnIfCOWDDkIejZ7h7glNPXzPxJ9HdbfwJJlhU3X0Xd7M9ackvU_LXPqELcTxd68iooH2sGYk0TKregqaRHc2prvNwENDxXnxTbcjIsELE0P4kf56bzunBit_v_-X6iA-SD0zVRoaweYP5D0rTncQoJL8UONCIbmj03iNuP-eQ6qiabF2vWDegn7mBiV2dKI_bI7poQtjJtFgm3WNxvMxucP6obBHQn5_6L90PgM3DYA5R6reaC7Ebh6OCBQtL9ms1vCNp0YBumjm40-FFit-4Ipuy0wi1lzXe56p1OBgPT_PB-eKvFEFYecIRZlYsx0rC8cIRZl5n6oMIY94kx37bcGqr5TarwoycHe9db2LI5k'

In [ ]:
title_query = TitleQuery(token)

In [ ]:
queried_data = title_query.get_instrument_data(instrument_type="cedears", country="argentina")

In [139]:
queried_df = pd.DataFrame(queried_data['titulos'])

In [140]:
# Expand the puntas column
queried_df[[key for key in queried_df['puntas'].iloc[0].keys()]] = queried_df['puntas'].apply(lambda x: pd.Series(x))
queried_df.head(2)

,simbolo,puntas,ultimoPrecio,variacionPorcentual,apertura,maximo,minimo,ultimoCierre,volumen,cantidadOperaciones,...,mercado,moneda,descripcion,plazo,laminaMinima,lote,cantidadCompra,precioCompra,precioVenta,cantidadVenta
0,AAL,"{'cantidadCompra': 2.0, 'precioCompra': 4880.5...",4929.5,1.18,4900.00,4998.5,4805.00,4929.5,3668.0,322.0,...,1,1,AMERICAN AIRLINES GROUP INC.,T2,1,1,2.0,4880.5,4924.5,1800.0
1,AALD,"{'cantidadCompra': 50.0, 'precioCompra': 6.9, ...",7.3,3.98,7.02,7.3,7.02,7.3,71.0,4.0,...,1,2,American Airlines Group Inc.,T2,1,1,50.0,6.9,7.4,1.0


In [141]:
queried_df = queried_df[['simbolo', 'ultimoPrecio', 'volumen', 'moneda', 'descripcion', 'plazo', 'precioVenta', 'precioCompra', 'cantidadCompra', 'cantidadVenta']].copy()
queried_df.head(2)

,simbolo,ultimoPrecio,volumen,moneda,descripcion,plazo,precioVenta,precioCompra,cantidadCompra,cantidadVenta
0,AAL,4929.5,3668.0,1,AMERICAN AIRLINES GROUP INC.,T2,4924.5,4880.5,2.0,1800.0
1,AALD,7.3,71.0,2,American Airlines Group Inc.,T2,7.4,6.9,50.0,1.0


In [142]:
# Create a df with the species in ARS
df_pesos = queried_df[queried_df['moneda'] == "1"].copy()

# Filter the species in dollars in dollar D (not cable)
letter = "D"
df_dolares = queried_df[(queried_df['moneda'] == "2") & (queried_df['simbolo'].str.endswith(letter))].copy()

In [143]:
# Join the dfs in ARS ans USD

# Add the name of the species in dollars to be able to join back into them
 # at one point it would be good to have the actual names of the species in dollars, because maybe they are different as the species name in pesos with a D in the end
df_pesos.loc[:,f'simbolo_{letter}'] = df_pesos["simbolo"] + letter

# Join them
df_pesos_and_dollars = pd.merge(df_pesos, df_dolares, left_on='simbolo_D', right_on="simbolo" ,how='left', suffixes=('', '_D'))

In [147]:
# Calculate the new columns MEP
df_pesos_and_dollars.loc[:, 'MEP'] = df_pesos_and_dollars['ultimoPrecio'] / df_pesos_and_dollars['ultimoPrecio_D']
df_pesos_and_dollars.loc[:, "MEP_venta_D"] = df_pesos_and_dollars["precioCompra"] / df_pesos_and_dollars["precioVenta_D"]
df_pesos_and_dollars.loc[:, "MEP_compra_P"] = df_pesos_and_dollars["precioVenta"] / df_pesos_and_dollars["precioCompra_D"]

In [150]:
# Drop rows with missing values in the 'MEP' column
df_mep = df_pesos_and_dollars.dropna(subset=['MEP']).copy()
df_mep = df_mep[df_mep.volumen > 0].copy()
df_mep = df_mep[df_mep.MEP < 9999].copy()

# Calculate the weighted average
mep_promedio_general = np.average(df_mep['MEP'], weights=df_mep['volumen'])
print("El promedio del MEP es: ", round(mep_promedio_general,1), "$")

El promedio del MEP es:  689.2 $


In [151]:
final_filter_volume = df_mep[(df_mep.volumen > 1000) & (df_mep.volumen_D > 100)]
final_filter_volume.sort_values(by="MEP", ascending=False)

,simbolo,ultimoPrecio,volumen,moneda,descripcion,plazo,precioVenta,precioCompra,cantidadCompra,cantidadVenta,...,moneda_D,descripcion_D,plazo_D,precioVenta_D,precioCompra_D,cantidadCompra_D,cantidadVenta_D,MEP,MEP_venta_D,MEP_compra_P
94,GOLD,12024.0,9032.0,1,Barrick Gold,T2,12028.5,12000.0,21.0,1000.0,...,2,Barrick Gold,T2,18.200,17.000,25.0,31.0,707.294118,659.340659,707.558824
143,MSFT,7993.0,14937.0,1,Microsoft,T2,7998.0,7975.0,3700.0,2973.0,...,2,Microsoft,T2,11.700,11.400,41.0,229.0,704.229075,681.623932,701.578947
140,MO,8052.0,2216.0,1,Altria Group,T2,8052.5,8030.0,162.0,2800.0,...,2,Altria Group,T2,11.550,11.450,7.0,7.0,703.231441,695.238095,703.275109
2,ABBV,11523.0,1449.0,1,AbbVie Inc.,T2,11523.0,11455.5,1000.0,1012.0,...,2,Abbvie Inc.,T2,16.900,16.200,36.0,5.0,702.621951,677.840237,711.296296
22,ARKK,2958.0,8764.0,1,Etf Ark Innovation,T2,2972.5,2955.0,27300.0,6310.0,...,2,Etf Ark Innovation,T2,4.500,4.160,50.0,50.0,702.612827,656.666667,714.543269
1,AAPL,13022.5,61531.0,1,Apple,T2,13052.0,13022.5,2200.0,1844.0,...,2,Apple,T2,18.850,18.550,73.0,35.0,702.021563,690.848806,703.611860
244,XOM,17240.0,5955.0,1,Exxon Mobil,T2,17206.5,17166.5,1400.0,299.0,...,2,Exxon Mobil,T2,25.500,24.500,2.0,10.0,700.813008,673.196078,702.306122
31,BABA,7006.0,21559.0,1,Alibaba Group,T2,7008.5,6990.0,2800.0,2800.0,...,2,Alibaba Group,T2,10.100,10.000,40.0,2.0,700.600000,692.079208,700.850000
117,INTC,5173.5,8012.0,1,Intel,T2,5198.0,5173.5,5800.0,422.0,...,2,Intel,T2,8.000,7.270,10.0,23.0,700.067659,646.687500,714.993122
137,META,9235.0,8273.0,1,Meta Platforms Inc,T2,9249.5,9225.0,4500.0,4500.0,...,2,Meta Platforms Inc,T2,13.600,12.500,24.0,16.0,699.621212,678.308824,739.960000


In [159]:
cols = ["simbolo","descripcion","volumen","ultimoPrecio", "precioVenta","precioCompra","cantidadCompra","cantidadVenta","volumen_D", "ultimoPrecio_D", "precioVenta_D","precioCompra_D","cantidadCompra_D","cantidadVenta_D","MEP", "MEP_venta_D", "MEP_compra_P"]
final_filter_volume[cols].to_csv("blaaa.csv", index=False)